# Automação Web e Busca de Informações com Python

In [30]:
!pip install selenium

In [31]:
#Passo 2 - Selenium - Criar um navegador
from selenium import webdriver #para criar o navegador
from selenium.webdriver.common.by import By #para localizar elementos
from selenium.webdriver.common.keys import Keys #Para enviar teclas

#Quando o webdriver estiver na mesmo local que o arquivo em python
navegador = webdriver.Chrome("chromedriver")

#Quando o webdriver estiver no local de instalação do python
#navegador = webdriver.Chrome()


In [32]:
#Passo 3 - Obter a cotação do dolar
navegador.get("https://google.com.br/")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("Cotação Dólar")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#Passo 4 - Obter a cotação do euro
navegador.get("https://google.com.br/")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("Cotação Euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#Passo 5 - Obter a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

5.5291
6.3338605050000005
323.68


### Agora vamos atualiza a base de preços com as novas cotações

- Importando a base de dados

In [33]:
#Passo 4 - Importar a base e atualizar as cotações
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [39]:
#Passo 5 - Atualizar os preços e atualizar/importar a base de dados
tabela.loc[tabela['Moeda'] == 'Dólar' , 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro' , 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro' , 'Cotação'] = float(cotacao_ouro)

#Atualizar as colunas

#Preço de compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#Preço de venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.529100,5529.044709,1.40,7740.662593
1,Carro Renault,4500.00,Euro,6.333861,28502.372273,2.00,57004.744545
2,Notebook Dell,899.99,Dólar,5.529100,4976.134709,1.70,8459.429005
3,IPhone,799.00,Dólar,5.529100,4417.750900,1.70,7510.176530
4,Carro Fiat,3000.00,Euro,6.333861,19001.581515,1.90,36103.004879
5,Celular Xiaomi,480.48,Dólar,5.529100,2656.621968,2.00,5313.243936
6,Joia 20g,20.00,Ouro,323.680000,6473.600000,1.15,7444.640000


### Agora vamos exportar a nova base de preços atualizada

In [41]:
tabela.to_excel("Produtos Novo.xlsx", index=False)